Step 1: Install Dependencies
mlflow.end_run()

In [1]:
# mlflow.end_run()

In [2]:
!pip install --upgrade pip
!pip install mlflow scikit-learn
!pip install ipynbname


  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
C:\Users\reema\anaconda3\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Step 2: Import Required Libraries


In [17]:
import os
import json
import hashlib
import platform
import subprocess
from datetime import datetime
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import mlflow.sklearn
import ipynbname
import subprocess
import ipynbname
import mlflow
from mlflow.tracking import MlflowClient

Step 3: Setup MLflow Experiment

In [4]:
# === Setup Experiment ===
mlflow.set_experiment("Iris_Model_Tracking") #TODO-- dynamically add name


<Experiment: artifact_location='file:///C:/Users/reema/OneDrive/Dokumente/Provenance_tracking_thesis/Provenence-Tracking-Thesis-Research/src/mlruns/385900191331244378', creation_time=1744549978614, experiment_id='385900191331244378', last_update_time=1744549978614, lifecycle_stage='active', name='Iris_Model_Tracking', tags={}>

Step 4: Define Utility Functions

In [5]:
# === Utility Functions ===
def get_git_info():
    try:
        commit = subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode().strip()
        branch = subprocess.check_output(['git', 'rev-parse', '--abbrev-ref', 'HEAD']).decode().strip()
        remote = subprocess.check_output(['git', 'config', '--get', 'remote.origin.url']).decode().strip()
        return commit, branch, remote
    except:
        return "N/A", "N/A", "N/A"

def hash_dataset(X):
    return hashlib.sha256(X.tobytes()).hexdigest()

def get_installed_packages():
    result = subprocess.check_output(["pip", "freeze"]).decode("utf-8")
    return result.splitlines()

# === Track Code Version ===
def get_code_version():
    try:
        # You can retrieve the latest commit or a specific tag for the code version.
        commit_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode().strip()
        return commit_hash
    except:
        return "N/A"


In [6]:
# get script path else assign github link
def get_script_path():
    try:
        # Try to get the notebook path
        nb_path = ipynbname.path()
        return str(nb_path)
    except:
        try:
            # Fallback to GitHub remote URL if in a git repo
            git_remote = subprocess.check_output(['git', 'config', '--get', 'remote.origin.url']).decode().strip()
            return git_remote
        except:
            return "N/A"

script_path = get_script_path()
print("Script Path:", script_path)


Script Path: https://github.com/reema-dass26/Provenence-Tracking-Thesis-Research.git


Step 5: Load Dataset and Split Data

In [7]:
# === Load Data ===
iris = datasets.load_iris()
X = iris.data
y = iris.target

# === Train/Test Split ===
SEED = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)


Step 6: Initialize and Train Model


In [8]:
# === Model Training ===
clf = RandomForestClassifier(n_estimators=100, max_depth=4, random_state=SEED)


Step 7: Start Logging with MLflow


In [22]:
# === Start Logging ===
with mlflow.start_run() as run:
    experiment_name = "Iris_Model_Experiment"
    mlflow.set_experiment(experiment_name)
    run_id = run.info.run_id
    
    # Track model and training process
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    # === Git Info ===
    git_commit, git_branch, git_remote = get_git_info()
    mlflow.set_tag("git_commit", git_commit)
    mlflow.set_tag("git_branch", git_branch)
    mlflow.set_tag("git_remote_url", git_remote)

# === Code Version ===
    code_version = get_code_version()  # Git commit hash or tag for code version
    mlflow.set_tag("code_version", code_version)


    # === Model & Data Info ===
    mlflow.set_tag("model_type", "RandomForestClassifier")
    mlflow.set_tag("dataset_name", "iris")
    mlflow.set_tag("dataset_hash", hash_dataset(X))
    mlflow.set_tag("run_start", str(datetime.now()))
    mlflow.set_tag("created_by", os.getenv("USER") or os.getenv("USERNAME"))

Exception: Run with UUID f3e4d3f2b7154a3b8661aef9fc93d02d is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True

In [10]:
    # === Parameters ===
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 4)
    mlflow.log_param("random_seed", SEED)
    mlflow.log_param("num_features", X.shape[1])
    mlflow.log_param("num_rows", X.shape[0])
    mlflow.log_param("train_test_split_ratio", 0.8)


0.8

In [11]:
    # === Environment Info ===
    mlflow.log_param("python_version", platform.python_version())
    mlflow.log_param("os_platform", platform.platform())
    
    installed_packages = get_installed_packages()
    with open("requirements_snapshot.txt", "w") as f:
        f.write("\n".join(installed_packages))
    mlflow.log_artifact("requirements_snapshot.txt")


In [12]:
    # === Metrics ===
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("precision", prec)
    mlflow.log_metric("recall", rec)
    mlflow.log_metric("f1_score", f1)


In [13]:
    # === Log Model ===
    mlflow.sklearn.log_model(clf, "model")


2025/04/13 15:41:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [24]:
experiment_name = "Iris_Model_Experiment"
mlflow.set_experiment(experiment_name)

# # Specify the model name
# model_name = "RandomForest_Iris_Model"

# # Instantiate the MlflowClient
# client = MlflowClient()

# # Fetch the latest version of the registered model
# latest_versions = client.get_latest_versions(model_name)

# # If versions exist, fetch the version
# if latest_versions:
#     model_registry_version = latest_versions[0].version
# else:
#     model_registry_version = "N/A"  # Default if the model is not found

# print("Model Registry Version:", model_registry_version)

# client = MlflowClient()
# registered_models = client.
# print("Registered Models:", [model.name for model in registered_models])


2025/04/13 15:46:18 INFO mlflow.tracking.fluent: Experiment with name 'Iris_Model_Experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/reema/OneDrive/Dokumente/Provenance_tracking_thesis/Provenence-Tracking-Thesis-Research/src/mlruns/749807049495519845', creation_time=1744551978713, experiment_id='749807049495519845', last_update_time=1744551978713, lifecycle_stage='active', name='Iris_Model_Experiment', tags={}>

In [27]:
from mlflow.tracking import MlflowClient

# Initialize the MLflow client
client = MlflowClient()

# Specify the model name
model_name = "RandomForest_Iris_Model"

# Check if the model is registered
registered_models = client.search_registered_models()

if registered_models:
    model = registered_models[0]  # Get the first registered model with the specified name
    print(f"Model {model_name} is registered. Model details:")
    print(f"Name: {model.name}, Creation Time: {model.creation_time}")
    
    # Now fetch the latest version of this model
    model_versions = client.get_model_versions(model_name)

    if model_versions:
        # Fetch the latest version
        latest_version = model_versions[-1].version
        print(f"Latest model version: {latest_version}")
    else:
        print("No versions available for the model.")
else:
    print(f"Model {model_name} is not found.")


Model RandomForest_Iris_Model is not found.


In [21]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Specify the model name
model_name = "RandomForest_Iris_Model"

# Fetch the model and its version
try:
    model = client.get_registered_model(model_name)
    # If the model is registered, you can fetch its latest version
    latest_versions = client.get_latest_versions(model_name)
    if latest_versions:
        model_registry_version = latest_versions[0].version
    else:
        model_registry_version = "N/A"
    print(f"Model Registry Version: {model_registry_version}")
except Exception as e:
    print(f"Error fetching model registry version: {e}")


Error fetching model registry version: Registered Model with name=RandomForest_Iris_Model not found


In [15]:
 # === Save Final Provenance JSON ===

provenance = {
        "run_id": run_id,
        "git_commit": git_commit,
        "git_branch": git_branch,
        "git_remote": git_remote,
        "dataset": "iris",
        "dataset_hash": hash_dataset(X),
        "model_type": "RandomForestClassifier",
        "parameters": {
            "n_estimators": 100,
            "max_depth": 4,
            "random_seed": SEED
        },
        "environment": {
            "python_version": platform.python_version(),
            "os_platform": platform.platform(),
            "packages": installed_packages
        },
        "metrics": {
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "f1_score": f1
        },
        "created_by": os.getenv("USER") or os.getenv("USERNAME"),
        "timestamp": str(datetime.now()),
        # Adding missing fields
        "experiment_name": mlflow.get_experiment_by_name("Iris_Model_Tracking").name,
        "run_name": run.info.run_name,  # Default run name or custom name
        "script_path": script_path,  # Path to the script
        "mlflow_source_name": mlflow.__file__,
        "model_registry_version":   # Placeholder for model registry version
    }

with open("run_provenance.json", "w") as f:
        json.dump(provenance, f, indent=4)

mlflow.log_artifact("run_provenance.json")

AttributeError: module 'mlflow' has no attribute 'registered_model_version'